# Universidad Oberta de Catalunya

## Práctica de Web Scraping - PRA1

### Integrantes del grupo:
   - Bryan Steven Cortez Chichande
   - Cesar Alexander Guzmán Vásquez

### Descripción del segundo modelo
El siguiente código permite realizar web scraping en la página Semantic Scholar con los siguientes temas:
   * Data Science
   * Data Mining
   * Web Mining
   * Web Scraping
   * Text Mining
   * Data Visualization

**1) Importación de librería utilizadas en las funciones creadas**

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import gc

Información utilizada para verificar el nivel de acceso en las búsquedas y la forma de acceder a la información.
    
    Verificación del archivo robot.txt
        https://www.semanticscholar.org/robots.txt
    Acceso a la API del sitio web para desarrolladores
        http://api.semanticscholar.org/
        https://api.semanticscholar.org/graph/v1#operation/get_graph_get_paper_search
        
**Nota. Se utilizó la API del sitio web Semantic Scholar, debido a que no se logró acceder a la estructura HTML mediante la URL de la misma.**

**2) Función utilizada para obtener los datos desde la página web de Semantic Scholar y guardarlos en un archivo csv**

In [ ]:
def get_data_of_api_semantic_scholar(keyword, limit, numpapers, csv_writer):
    """
        Parámetros de entrada:
            keyword    String: Término de búsqueda
            limit      int:    Número de documentos por búsqueda
            numpapers  int:    Número total de artículos que se desea obtener
            csv_writer Object: Objeto relacionado al archivo csv
    """
    
    _valoffset=0
    _vallimit=limit
    _valnumarticulos=numpapers
    try:
        
        while _valnumarticulos>0:
            _urlwbsc="http://api.semanticscholar.org/graph/v1/paper/search?query="+str(keyword)+"&offset="+str(_valoffset)+"&limit="+str(_vallimit)+"&fields=paperId,title,authors,url,year,referenceCount,citationCount,influentialCitationCount,isOpenAccess,fieldsOfStudy"
            _webpagesc=requests.get(_urlwbsc)
            _webpagesc.raise_for_status()
            _resultwp = _webpagesc.json()
            _numrows=len(_resultwp["data"])
            _contador=0

            while _contador<_numrows:
                _paper=_resultwp["data"][_contador]
                _datapp=[str(_paper["paperId"]),str(_paper["title"]),_paper["year"],_paper["url"],_paper["referenceCount"],
                         _paper["citationCount"],_paper["influentialCitationCount"],_paper["isOpenAccess"]]
                csv_writer.writerow(_datapp)

                _contador+=1
            
            gc.collect();

            _valoffset+=(_vallimit+1)
            _valnumarticulos-=_vallimit

    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)    
    

**3) Función principal de la búsqueda, captura y recopilación de los datos bibliográficos de documentos científicos**

In [ ]:
def get_data_of_documents(limits, numpapers):
    """
        Parámetros de entrada:
            limits     int: Número de documentos por búsqueda
            numpapers  int: Número total de artículos que se desea obtener
    """
    
    _listpapers=[]
    _datahead = ['PaperId', 'Title', 'Year', 'URL', 'ReferenceCount', 'CitationCount', 'InfluentialCitationCount', 'IsOpenAccess']
    _listkeywords=['data%20science','data%20mining','web%20mining','web%20scraping','text%20mining','data%20visualization']
    
    csv_file =  open("_dtpapers.csv", 'w', newline='', encoding="utf-8")
    csv_writer = csv.writer(csv_file, delimiter=",", quoting=csv.QUOTE_ALL)
    csv_writer.writerow(_datahead)
    
    for _keyword in _listkeywords:
        get_data_of_api_semantic_scholar(_keyword, limits, numpapers, csv_writer);

    csv_file.close()
    

**Llamada a la función principal**

In [ ]:
get_data_of_documents(100,10000);